In [9]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, JsonOutputToolsParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.output_parsers import JsonOutputParser

In [2]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from pydantic import BaseModel, Field
from typing import List
import pandas as pd
from pydantic.config import ConfigDict

from langchain_openai import OpenAI
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
class Autograph(BaseModel):
    graph_needed: bool = Field(
        ..., description="For the answer produced by the CSV-agent for the user's query can a graph be constructed ? Give output either True or False"
    )
    code: str = Field(
        ..., description="Output code to be used to render the graph using python and plotly"
    )
    

parser = JsonOutputParser(pydantic_object=Autograph)

In [3]:
# open_ai_api =""
# os.environ['OPENAI_API_KEY'] = open_ai_api
# model = ChatOpenAI(temperature=0, model="gpt-4")

In [ ]:
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY") 
aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

llm = AzureChatOpenAI(
    model="gpt-4o",
    deployment_name="gpt-4o",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)


# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

In [4]:
agent_question = "Give me percentage distribution of department types within the firm"
agent_answer = "Here is the percentage distribution of department types within the firm: Production: 67.20%,IT/IS: 16.08% Sales: 9.97% Software Engineering: 3.54% Admin Offices: 2.89% Executive Office: 0.32%"

template = """
Your task is to convert CSV-agent's answer into plotly graph object incapsulated into a function which does not take any variable as input, if possible.
Using Python version 3.9.12, create a script using the dataframe df to graph the following: Ensure that the code that you are crafting in confined within the function create_plotly_figure(),which takes no postional argument.
Refer to the following example below where it is showcased how an answer of the agent is first determined if it can be graphed. 
Just return the function create_plotly_figure() dont call the function, and a Boolean variable True or False depending if a graph can be made.
Always employ various colors to showcase difference in the plot and when approriate also use "hue" to make the graphs more meaningful                

Example:

Input 1: user_question: Give me count of emplyment status within the firm
Input 2: agent_answer: The count of employment status within the firm is as follows:

Active: 207 employees
Voluntarily Terminated: 88 employees
Terminated for Cause: 16 employees 

Output 1: True
Output 2: 

def create_plotly_figure():
    employment_status = np.array(["Active", "Voluntarily Terminated", "Terminated for Cause"])
    counts = np.array([207, 88, 16])

    # Creating a DataFrame
    df = pd.DataFrame({{'EmploymentStatus': employment_status, 'Count': counts}})

    
    fig = go.Figure(data=[go.Bar(x=df['EmploymentStatus'], y=df['Count'])])
    fig.update_layout(title_text="Count of Employment Status",
                      xaxis_title="Employment Status",
                      yaxis_title="Count") 

    
    return fig 
Note that in output 2 you only have to give the code snipet nothing else

Current Task: Please refer the the pervious examples and commands to produce a desired output

Input 1: {agent_question}
Input 2: {agent_answer}

Format Requirement:
{format_instructions}
                       
Answer:

"""

In [5]:
prompt = PromptTemplate(
    template= template,
    input_variables= ["agent_question", "agent_answer"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [6]:
chain = prompt | model | parser

answer = chain.invoke({"agent_question": agent_question ,"agent_answer": agent_answer})

In [7]:
answer

{'graph_needed': True,
 'code': "def create_plotly_figure():\n    import plotly.graph_objects as go\n    import pandas as pd\n    import numpy as np\n\n    department_types = np.array(['Production', 'IT/IS', 'Sales', 'Software Engineering', 'Admin Offices', 'Executive Office'])\n    percentages = np.array([67.20, 16.08, 9.97, 3.54, 2.89, 0.32])\n\n    # Creating a DataFrame\n    df = pd.DataFrame({'DepartmentTypes': department_types, 'Percentage': percentages})\n\n    fig = go.Figure(data=[go.Pie(labels=df['DepartmentTypes'], values=df['Percentage'], hole=.3)])\n    fig.update_layout(title_text='Percentage Distribution of Department Types')\n\n    return fig"}

In [8]:
answer['code']

"def create_plotly_figure():\n    import plotly.graph_objects as go\n    import pandas as pd\n    import numpy as np\n\n    department_types = np.array(['Production', 'IT/IS', 'Sales', 'Software Engineering', 'Admin Offices', 'Executive Office'])\n    percentages = np.array([67.20, 16.08, 9.97, 3.54, 2.89, 0.32])\n\n    # Creating a DataFrame\n    df = pd.DataFrame({'DepartmentTypes': department_types, 'Percentage': percentages})\n\n    fig = go.Figure(data=[go.Pie(labels=df['DepartmentTypes'], values=df['Percentage'], hole=.3)])\n    fig.update_layout(title_text='Percentage Distribution of Department Types')\n\n    return fig"